In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# 현재경로 확인
os.getcwd()

'C:\\실습자료\\part4'

In [3]:
# 데이터 불러오기
data = pd.read_csv("./data/otto_train.csv") # Product Category
data.head() # 데이터 확인

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [4]:
'''
id: 고유 아이디
feat_1 ~ feat_93: 설명변수
target: 타겟변수 (1~9)
'''

'\nid: 고유 아이디\nfeat_1 ~ feat_93: 설명변수\ntarget: 타겟변수 (1~9)\n'

In [5]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 61878 nVar: 95


## 의미가 없다고 판단되는 변수 제거

In [6]:
data = data.drop(['id'], axis = 1) # id 제거

## 타겟 변수의 문자열을 숫자로 변환

In [7]:
mapping_dict = {"Class_1": 1,
                "Class_2": 2,
                "Class_3": 3,
                "Class_4": 4,
                "Class_5": 5,
                "Class_6": 6,
                "Class_7": 7,
                "Class_8": 8,
                "Class_9": 9}
after_mapping_target = data['target'].apply(lambda x: mapping_dict[x])

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [8]:
feature_columns = list(data.columns.difference(['target'])) # target을 제외한 모든 행
X = data[feature_columns] # 설명변수
y = after_mapping_target # 타겟변수
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(49502, 93) (12376, 93) (49502,) (12376,)


## 1. XGBoost

In [9]:
# !pip install xgboost
import xgboost as xgb
import time
start = time.time() # 시작 시간 지정
xgb_dtrain = xgb.DMatrix(data = train_x, label = train_y) # 학습 데이터를 XGBoost 모델에 맞게 변환
xgb_dtest = xgb.DMatrix(data = test_x) # 평가 데이터를 XGBoost 모델에 맞게 변환
xgb_param = {'max_depth': 10, # 트리 깊이
         'learning_rate': 0.01, # Step Size
         'n_estimators': 100, # Number of trees, 트리 생성 개수
         'objective': 'multi:softmax', # 목적 함수
        'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
xgb_model_predict = xgb_model.predict(xgb_dtest) # 평가 데이터 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, xgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Accuracy: 76.67 %
Time: 5.90 seconds


In [10]:
xgb_model_predict

array([5., 3., 6., ..., 9., 2., 7.], dtype=float32)

## 2. LightGBM

In [11]:
# !pip install lightgbm
import lightgbm as lgb
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'objective': 'multiclass', # 목적 함수
            'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, lgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Accuracy: 73.57 %
Time: 6.08 seconds


In [12]:
lgb_model.predict(test_x)

array([[9.14195396e-16, 2.27101123e-02, 3.85432853e-01, ...,
        3.25763637e-02, 7.61804989e-02, 4.64683926e-02],
       [1.16264426e-15, 3.77262272e-02, 2.26739830e-01, ...,
        1.92812290e-01, 1.01826669e-01, 8.11988908e-02],
       [8.03061518e-16, 1.74144816e-02, 1.18616633e-01, ...,
        2.57008295e-02, 6.69196402e-02, 4.08194769e-02],
       ...,
       [8.35633463e-16, 5.03899675e-02, 1.67414994e-01, ...,
        4.81925245e-02, 1.02188157e-01, 3.95359380e-01],
       [1.01560525e-15, 2.13472780e-02, 4.60509516e-01, ...,
        3.32864538e-02, 8.46310485e-02, 5.16230380e-02],
       [8.92775603e-16, 2.04756016e-02, 1.31867900e-01, ...,
        4.13697353e-01, 1.44381292e-01, 4.53796283e-02]])

## 3. Catboost

In [13]:
# !pip install catboost
import catboost as cb
start = time.time() # 시작 시간 지정
cb_dtrain = cb.Pool(data = train_x, label = train_y) # 학습 데이터를 Catboost 모델에 맞게 변환
cb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'eval_metric': 'Accuracy', # 평가 척도
            'loss_function': 'MultiClass'} # 손실 함수, 목적 함수
cb_model = cb.train(pool = cb_dtrain, params = cb_param) # 학습 진행
cb_model_predict = np.argmax(cb_model.predict(test_x), axis = 1) + 1 # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측, 인덱스의 순서를 맞추기 위해 +1
print("Accuracy: %.2f" % (accuracy_score(test_y, cb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

0:	learn: 0.6114702	total: 476ms	remaining: 47.2s
1:	learn: 0.6235304	total: 910ms	remaining: 44.6s
2:	learn: 0.6410246	total: 1.4s	remaining: 45.1s
3:	learn: 0.6410650	total: 1.85s	remaining: 44.5s
4:	learn: 0.6442770	total: 2.3s	remaining: 43.7s
5:	learn: 0.6455901	total: 2.75s	remaining: 43s
6:	learn: 0.6468224	total: 3.21s	remaining: 42.6s
7:	learn: 0.6484991	total: 3.65s	remaining: 42s
8:	learn: 0.6512666	total: 4.11s	remaining: 41.5s
9:	learn: 0.6491859	total: 4.55s	remaining: 40.9s
10:	learn: 0.6517716	total: 4.97s	remaining: 40.2s
11:	learn: 0.6526807	total: 5.39s	remaining: 39.5s
12:	learn: 0.6531655	total: 5.83s	remaining: 39s
13:	learn: 0.6537918	total: 6.27s	remaining: 38.5s
14:	learn: 0.6566603	total: 6.71s	remaining: 38s
15:	learn: 0.6563573	total: 7.15s	remaining: 37.5s
16:	learn: 0.6582966	total: 7.59s	remaining: 37.1s
17:	learn: 0.6591047	total: 8.02s	remaining: 36.6s
18:	learn: 0.6597309	total: 8.47s	remaining: 36.1s
19:	learn: 0.6603572	total: 8.9s	remaining: 35.6s
2

In [14]:
cb_model.predict(test_x)

array([[-0.34047199,  1.18951783,  0.4296545 , ..., -0.17528062,
        -0.02512535, -0.23089952],
       [-0.05653916,  0.45365426,  0.17195659, ...,  0.22287606,
         0.26190632,  0.18447355],
       [-0.31311   , -0.31539546, -0.31876501, ..., -0.29816626,
        -0.238871  , -0.3180434 ],
       ...,
       [ 0.04726952,  0.03794315, -0.17880234, ..., -0.2456077 ,
         0.11886516,  1.54034182],
       [-0.55746055,  1.77202298,  0.97862619, ..., -0.33775734,
        -0.48741853, -0.37792919],
       [-0.31665504,  0.08335556, -0.10060578, ...,  0.67896862,
         1.03198771, -0.14349506]])

In [15]:
# 데이터 불러오기
data = pd.read_csv("./data/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [16]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

In [20]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


In [21]:
# !pip install lightgbm
import lightgbm as lgb
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 500, # Number of trees, 트리 생성 개수
            'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [22]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sqrt(mean_squared_error(lgb_model.predict(test_x),test_y))

210904.17249451784

## Ensemble의 Ensemble

In [28]:
import random
bagging_predict_result = [] # 빈 리스트 생성
for _ in range(30):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    print(len(set(random_data_index)))
    lgb_dtrain = lgb.Dataset(data = train_x.iloc[random_data_index,], label = train_y.iloc[random_data_index]) # 학습 데이터를 LightGBM 모델에 맞게 변환
    lgb_param = {'max_depth': 10, # 트리 깊이
                'learning_rate': 0.01, # Step Size
                'n_estimators': 500, # Number of trees, 트리 생성 개수
                'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
    lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
 
    predict1 = lgb_model.predict(test_x) # 테스트 데이터 예측
    bagging_predict_result.append(predict1) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장
    print(sqrt(mean_squared_error(lgb_model.predict(test_x),test_y)))
    

9498


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


213840.28212887075
9588
215281.48865139345
9541
216931.65773132836
9520
210802.66886492126
9618
219071.64304094203
9667
216314.75756279973
9572
213490.6784153256
9510
214367.47843199354
9559
215167.35603033012
9514
213894.60116466266
9586
215542.21615099907
9594
216843.20340590688
9587
218798.8520195591
9578
214293.95771643083
9553
213108.89210555324
9565
211831.95467274872
9545
213346.05009755393
9539
213389.82737112517
9501
212631.08537300932
9566
213590.2925155159
9582
213012.91050203642
9555
218171.88517125716
9558
214693.27509314194
9580
215236.73024988046
9473
212401.11461307958
9588
214333.78269783655
9533
211302.86203613353
9584
213408.6217110172
9593
217056.41671631008
9560
216769.20065211717


In [29]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성
for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [30]:
bagging_predict

[505315.6431804939,
 633564.7588044847,
 952766.3675341519,
 1592852.2628952032,
 639031.8368375365,
 371737.0954080787,
 707070.0475827972,
 429693.9067266107,
 462671.96190782986,
 492547.4905826759,
 636396.1473618048,
 381261.2503278578,
 298628.1803960091,
 359640.1123279075,
 343752.5023386434,
 1322055.8680471545,
 374558.9792837914,
 1051259.9082893652,
 317781.39315019635,
 526503.4083127563,
 377727.5012286073,
 1953443.1564075807,
 660749.5206424318,
 539067.5540566791,
 499679.9533561105,
 488138.7817814946,
 295696.922306135,
 255149.85825554014,
 475632.90956849256,
 532801.4288177342,
 497781.1207777062,
 467815.1517221897,
 467565.8846161754,
 588651.8627834369,
 381800.0802840279,
 1042467.9412875155,
 896531.1592280201,
 531235.3118472626,
 358923.2710820017,
 1562352.0299626426,
 394606.0308845912,
 275317.7087512416,
 509264.5286560714,
 342563.2384020289,
 251747.56835750202,
 243257.8427399587,
 334404.73407011916,
 334179.6504792845,
 356738.3508694054,
 567413.3

In [31]:
sqrt(mean_squared_error(bagging_predict,test_y))

209736.90720982864